In [22]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [46]:
ds = ImageDataGenerator(rotation_range=20, 
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        validation_split=0.2,
                        rescale=1/255,)

In [78]:
train_dataset = ds.flow_from_directory("Datasets/Equation Solver Dataset-small", 
                                 class_mode='categorical', color_mode = "grayscale",
                                 target_size=(45,45),
                                 subset="training")

test_dataset = ds.flow_from_directory("Datasets/Equation Solver Dataset-small",
                                 class_mode='categorical', color_mode = "grayscale",
                                 target_size=(45,45),
                                 subset="validation")

Found 181738 images belonging to 22 classes.
Found 45423 images belonging to 22 classes.


In [79]:
#Exploring Dataset

In [80]:
print(train_dataset)

In [81]:
train_dataset.class_indices

{'+': 0,
 '-': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11,
 '=': 12,
 'X': 13,
 'cos': 14,
 'div': 15,
 'log': 16,
 'sin': 17,
 'sqrt': 18,
 'tan': 19,
 'times': 20,
 'y': 21}

In [82]:
import cv2
cv2.imread("Datasets/Equation Solver Dataset-small/-/-_66.jpg", cv2.IMREAD_GRAYSCALE).shape

(45, 45)

In [83]:
#Defining Model

In [84]:
model = tf.keras.models.Sequential([
    #1st Layer
    tf.keras.layers.BatchNormalization(name="BN1",input_shape=(45,45,1),),
    tf.keras.layers.ZeroPadding2D(name="ZP1"),
    tf.keras.layers.Conv2D(32,(3,3),activation="relu",padding="valid",name='Conv1',),
    tf.keras.layers.ZeroPadding2D(name="ZP2"),
    tf.keras.layers.Conv2D(32,(3,3),activation="relu",padding="valid",name='Conv2'),
    tf.keras.layers.MaxPool2D((2,2),name="MP1"),
    #2nd Layer
    tf.keras.layers.BatchNormalization(name="BN2"),
    tf.keras.layers.ZeroPadding2D(name="ZP3"),
    tf.keras.layers.Conv2D(64,(3,3),activation="relu",padding="valid",name="Conv3"),
    tf.keras.layers.ZeroPadding2D(name="ZP4"),
    tf.keras.layers.Conv2D(64,(3,3),activation="relu",padding="valid",name='Conv4'),
    tf.keras.layers.MaxPool2D((2,2),name="MP2"),
    #3rd Layer
    tf.keras.layers.BatchNormalization(name="BN3"),
    tf.keras.layers.ZeroPadding2D(name="ZP5"),
    tf.keras.layers.Conv2D(128,(3,3),activation="relu",padding="valid",name="Conv5"),
    tf.keras.layers.ZeroPadding2D(name="ZP6"),
    tf.keras.layers.Conv2D(128,(3,3),activation="relu",padding="valid",name='Conv6'),
    tf.keras.layers.MaxPool2D((2,2),name="MP3"),
    #Flat Layers
    tf.keras.layers.BatchNormalization(name="BN4"),
    tf.keras.layers.Flatten(name="Flatten"),
    tf.keras.layers.Dense(256,activation="relu",name="Dense1"),
    tf.keras.layers.Dense(512,activation="relu",name="Dense2"),
    tf.keras.layers.BatchNormalization(name="BN5"),
    tf.keras.layers.Dense(22,activation="softmax",name="Dense3")
])

In [85]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BN1 (BatchNormalization)     (None, 45, 45, 1)         4         
_________________________________________________________________
ZP1 (ZeroPadding2D)          (None, 47, 47, 1)         0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 45, 45, 32)        320       
_________________________________________________________________
ZP2 (ZeroPadding2D)          (None, 47, 47, 32)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 45, 45, 32)        9248      
_________________________________________________________________
MP1 (MaxPooling2D)           (None, 22, 22, 32)        0         
_________________________________________________________________
BN2 (BatchNormalization)     (None, 22, 22, 32)       

In [86]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [87]:
history = model.fit(train_dataset,epochs=3,verbose=1,batch_size=32,validation_data=test_dataset)

Epoch 1/3
5680/5680 [==============================] - 4761s 838ms/step - loss: 0.5185 - accuracy: 0.8514 - val_loss: 0.1602 - val_accuracy: 0.9493
Epoch 2/3
5680/5680 [==============================] - 4938s 869ms/step - loss: 0.1643 - accuracy: 0.9476 - val_loss: 0.1556 - val_accuracy: 0.9539
Epoch 3/3
5680/5680 [==============================] - 27713s 5s/step - loss: 0.1252 - accuracy: 0.9589 - val_loss: 0.0948 - val_accuracy: 0.9672


In [88]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
